In [153]:
import pandas as pd
from geopy.geocoders import ArcGIS

In [154]:
# read initial csv file
df=pd.read_csv("scrapedPages.csv")

In [155]:
# create geocoder instance to test geocoding service
geoLocator=ArcGIS()
location=geoLocator.geocode("1003 Winchester Blvd. Rock Springs, WY 82901")
#dir(location)
location.latitude

41.58501000000007

In [165]:
# create a new column which combines the address and locality for ease of geocoding and remove irrelevant columns
def getAddress(add,local):
    return add+" "+local

df["Address"]=[getAddress(add,local) for add,local in zip(df["Address"],df["Locality"])]
df2=df.drop(columns=["Locality","Lot Size","Half Baths","Unnamed: 0", "Full Baths"],axis=1)
df2.to_csv("forGeocoding.csv",index=False)

In [166]:
# read the newly generated csv
df2=pd.read_csv("forGeocoding.csv")

# function to geocode addresses supplied
def getLocation(address):
    geoLocator=ArcGIS()
    location=geoLocator.geocode(address)
    return location

# Test
example_address=df2.loc[1,'Address'] # loc method specifies row index and col
getLocation(example_address)

Location(82901, Rock Springs, Wyoming, (41.58501000000007, -109.21828999999997, 0.0))

In [ ]:
# create two new columns in dataframe and use getLocation func to populate them via list comprehensions
df2["Latitude"]=[getLocation(address).latitude for address in df2["Address"]]
df2["Longitude"]=[getLocation(address).longitude for address in df2["Address"]]
print(df2)

In [173]:
# for production - to check that one of the columns of the uploaded file is named as either 'address' or 'Address'
def checkFileColumns(df):
    if "address" in df.columns or "address".title() in df.columns:
        return True    

# Test if it works for column with 'addresss'
df3=df2.rename(columns={"Address":"address"})
checkFileColumns(df3)

True

In [174]:
if checkFileColumns(df3) is True:
    print("just testing")

just testing


In [175]:
# create file without address column for testing
df4=df3.drop(columns="address",axis=1)
df4.to_csv("noAddress.csv",index=False)